In [42]:
import argparse
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW
from sklearn.model_selection import train_test_split
import nltk

from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
from nltk.translate.meteor_score import meteor_score
#from rouge import Rouge

from torch import cuda
from tqdm import tqdm
import numpy as np

device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [43]:
#!pip install git+https://github.com/alirezamshi/RQUGE

In [44]:
import pickle
with open("ennlarged_eval.pickle", "rb") as fp:
    enlarged_eval = pickle.load(fp)

with open("enlarged_finetune.pickle", "rb") as fp:
    enlarged_finetune = pickle.load(fp)
import random
random.seed(0)
random.shuffle(enlarged_finetune)



In [45]:
# New dataset with only the first question from each entry
new_dataset = [[context, answer, [questions[0]]] for context, answer, questions in enlarged_finetune]

# Print the new dataset
for entry in new_dataset:
    print(entry)
    break
    
# New dataset with only the first question from each entry
eval_new_dataset = [[context, answer, [questions[0]]] for context, answer, questions in enlarged_eval]

# Print the new dataset
for entry in eval_new_dataset:
    print(entry)
    break

['Next Unit of Computing (NUC) is a small form factor PC designed by Intel. Its first generation is based on the Sandy Bridge Celeron CPU. Its second generation CPU is based on the Ivy Bridge Core i3 and Core i5 processors. Its third generation is based on the Haswell architecture. Intel does offer for sale just the NUC motherboards, which have a built-in CPU, although (as of 2013) the price of a NUC motherboard is very close to the corresponding cased kit; third-party cases for the NUC boards are also available.', 'Is a small form factor PC designed by Intel. Its first generation is based on the Sandy Bridge Celeron CPU.', ['what is a NUc']]
['Purebred dogs of one breed are genetically distinguishable from purebred dogs of other breeds, but the means by which kennel clubs classify dogs is unsystematic. Systematic analyses of the dog genome has revealed only four major types of dogs that can be said to be statistically distinct. These include the "old world dogs" (e.g., Malamute and Sh

In [46]:
train_dataset = {'question':[], 'answer':[], 'context':[]}
for data in new_dataset:
    for question in data[2]:
        train_dataset['answer'].append(data[1])
        train_dataset['context'].append(data[0])
        train_dataset['question'].append(question)
        
import pandas as pd
dataframe = pd.DataFrame(train_dataset)
dataframe['ctxa'] = 'answer : ' + dataframe['answer'] + ' context : ' + dataframe['context']
main_dataset = dataframe[:5000]

eval_dataset = {'question':[], 'answer':[], 'context':[]}
for data in eval_new_dataset:
    for question in data[2]:
        eval_dataset['answer'].append(data[1])
        eval_dataset['context'].append(data[0])
        eval_dataset['question'].append(question)
        
import pandas as pd
dataframe2 = pd.DataFrame(eval_dataset)
dataframe2['ctxa'] = 'answer : ' + dataframe2['answer'] + ' context : ' + dataframe2['context']
validation_data = pd.DataFrame(dataframe2)
valid_data = pd.DataFrame(dataframe2)

In [47]:
main_dataset

,question,answer,context,ctxa
0,what is a NUc,Is a small form factor PC designed by Intel. I...,Next Unit of Computing (NUC) is a small form f...,answer : Is a small form factor PC designed by...
1,What sorts of expectations can lead to stress ...,expectations to substitute,Teachers face several occupational hazards in ...,answer : expectations to substitute context : ...
2,What is another term for Middle Persian?,Pahlavi,Middle Persian (Pahlavi) was the official lang...,answer : Pahlavi context : Middle Persian (Pah...
3,how long should you eat after a workout,15 minutes,To replace the glycogen stores in your body qu...,answer : 15 minutes context : To replace the g...
4,what's the average cost of a conservatory,"£5,000 and up mark.",The average quote for a typical conservatory t...,"answer : £5,000 and up mark. context : The ave..."
...,...,...,...,...
4995,Where doesn't the chloroplast want to carry ou...,the cytosol,After a chloroplast polypeptide is synthesized...,answer : the cytosol context : After a chlorop...
4996,what is a succulent plant,"Succulent plants, such as this Aloe, store wat...","Succulent plants, such as this Aloe, store wat...","answer : Succulent plants, such as this Aloe, ..."
4997,When did Beyonce start becoming popular?,in the late 1990s,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,answer : in the late 1990s context : Beyoncé G...
4998,Which of the following is not a pigment: phyco...,Cyanobacteria,Cyanobacteria are considered the ancestors of ...,answer : Cyanobacteria context : Cyanobacteria...


In [48]:
validation_data

,question,answer,context,ctxa
0,The methods kennel clubs used to classify dogs...,unsystematic.,Purebred dogs of one breed are genetically dis...,answer : unsystematic. context : Purebred dogs...
1,What is another term for southern Europe,Mediterranean Europe,"Some definitions of southern Europe, also know...",answer : Mediterranean Europe context : Some d...
2,Who wrote the only book length- biography of H...,Charles Shields,"Charles Shields, who has written the only book...",answer : Charles Shields context : Charles Shi...
3,When did the BBC Natural History Unit come int...,the 1950s,David Attenborough was later granted sabbatica...,answer : the 1950s context : David Attenboroug...
4,What does the bacteria give the worm?,nutrition,One of the most spectacular examples of obliga...,answer : nutrition context : One of the most s...
...,...,...,...,...
1495,Tetzel was known for a?,saying,"However, this oft-quoted saying of Tetzel was ...","answer : saying context : However, this oft-qu..."
1496,Why were some people unhappy about the exterio...,views of the city were blocked,The UChicago Arts program joins academic depar...,answer : views of the city were blocked contex...
1497,What turbines run double the speed of nuclear ...,electric,A steam turbine consists of one or more rotors...,answer : electric context : A steam turbine co...
1498,What needs to be saved?,steam,The simplest valve gears give events of fixed ...,answer : steam context : The simplest valve ge...


In [49]:


class QGDataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = 500  # Fixed source length
        self.target_len = 40   # Fixed target length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        ctxa = self.data.iloc[index]['ctxa']
        question = self.data.iloc[index]['question']
        source_encoded = self.tokenizer(ctxa, max_length=self.source_len, truncation=True, padding="max_length", return_tensors="pt")
        target_encoded = self.tokenizer(question, max_length=self.target_len, truncation=True, padding="max_length", return_tensors="pt")
        return {
            'input_ids': source_encoded['input_ids'].squeeze(0),
            'attention_mask': source_encoded['attention_mask'].squeeze(0),
            'labels': target_encoded['input_ids'].squeeze(0)
        }

In [50]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large').to(device)

dataset = QGDataset(main_dataset, tokenizer)
loader = DataLoader(dataset, batch_size=1, shuffle=True)

optimizer = AdamW(model.parameters(), lr=5e-5)

/home/erap/.local/lib/python3.9/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [51]:


def train_epoch(model, data_loader, optimizer):
    model.train()
    total_loss = 0
    # Wrap the data_loader with tqdm for a progress bar
    for batch in tqdm(data_loader, desc="Training Progress"):
        inputs = {key: value.to(device) for key, value in batch.items()}
        outputs = model(**inputs)
        loss = outputs.loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(data_loader)

In [52]:
# for epoch in range(3):
#     loss = train_epoch(model, loader, optimizer)
#     print(f"Epoch {epoch + 1}, Loss: {loss:.2f}")

for epoch in range(3):
    loss = train_epoch(model, loader, optimizer)
    print(f"Epoch {epoch + 1}, Loss: {loss:.2f}")

Training Progress: 100%|██████████| 5000/5000 [10:45<00:00,  7.75it/s]


Epoch 1, Loss: 2.32


Training Progress: 100%|██████████| 5000/5000 [10:43<00:00,  7.77it/s]


Epoch 2, Loss: 2.32


Training Progress: 100%|██████████| 5000/5000 [10:42<00:00,  7.78it/s]

Epoch 3, Loss: 2.26


In [53]:
model.save_pretrained('./final4/')
print("Training complete and model saved.")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


Training complete and model saved.


In [54]:
# save model

In [55]:
# model evaluation

In [56]:
class validate_Dataset(Dataset):
    def __init__(self, dataframe, tokenizer, source_len, max_target_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len  
        self.max_target_len = max_target_len 

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        ctxa = str(self.data.iloc[index]['ctxa'])
        question = str(self.data.iloc[index]['question'])
        #print("Context:", ctxa)  # Debug print
        #print("Question:", question)  # Debug print
        source_encoded = self.tokenizer(ctxa, max_length=self.source_len, truncation=True, padding="max_length", return_tensors="pt")
        target_encoded = self.tokenizer(question, max_length=self.max_target_len, truncation=True, padding="max_length", return_tensors="pt")
        return {
            'source_ids': source_encoded['input_ids'].squeeze(0),
            'source_mask': source_encoded['attention_mask'].squeeze(0),
            'labels': target_encoded['input_ids'].squeeze(0)
        }


In [57]:
def validate(tokenizer, model, device, loader):
    model.eval()
    predictions = []
    questions = []
    with torch.no_grad():
        for data in tqdm(loader, desc="Validating"):
            input_ids = data['source_ids'].to(device)
            attention_mask = data['source_mask'].to(device)
            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=70,
                num_beams=5,
                no_repeat_ngram_size=2,
                temperature=0.9,  # Adjust temperature to make it less likely to pick low probability options
                top_k=50,         # Limits to the top 50 choices
                top_p=0.95,       # Cumulative probability cutoff
                early_stopping=True
            )
           
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            actual_questions = [tokenizer.decode(l, skip_special_tokens=True) for l in data['labels']]
            
            predictions.extend(preds)
            questions.extend(actual_questions)

    return predictions, questions



In [58]:

# Load the tokenizer and model

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model_checkpoint = './final4/'  
model = BartForConditionalGeneration.from_pretrained(model_checkpoint).to(device)

In [59]:
# step 4
# Assuming the data is already loaded and split into training and validation sets
validation_data.reset_index(drop=True, inplace=True)
validation_set = validate_Dataset(validation_data, tokenizer, source_len=500, max_target_len=50)
validation_loader = DataLoader(validation_set, batch_size=8, shuffle=False, num_workers=2)


In [60]:
predictions, actual_questions = validate(tokenizer, model, device, validation_loader)
print("Predictions:", predictions)
print("Actual Questions:", actual_questions)

Validating:   0%|          | 0/188 [00:00<?, ?it/s]/home/erap/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/erap/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Validating: 100%|██████████| 188/188 [05:45<00:00,  1.84s/it]

Predictions: ['WhatWhichwhatWho ishowHowInWhenWherewhere didThe was theOnwhenWhy long does much of manyTowhich arewhyAAccording what a kindOf', 'WhatWhichwhatWho ishowHowInWhenWherewhere didThe was theOnwhenWhy long does much of manyTowhich arewhyAAccording what a kindOf', 'WhatWhichwhatWho ishowHowInWhenWherewhere didThe was theOnwhenWhy long does much of manyTowhich arewhyAAccording what a kindOf', 'whatWhatWhichWho ishowHowInWhenWherewhere didThe was theOnwhenWhy long does much of manyTowhich arewhyAAccording what a kindOf', 'WhatwhatWhichWho ishowHowInWhenWherewhere didThe was theOnwhenWhy long does much of manyTowhich arewhyAAccording what a kindOf', 'WhatwhatWhichWho ishowHowInWhenWherewhere didThe was theOnwhenWhy long does much of manyTowhich arewhyAAccording what a kindOf', 'WhatWhichwhatWho ishowHowInWhenWherewhere didThe was theOnwhenWhy long does much of manyTowhich arewhyAAccording what a kindOf', 'whatWhatWhichWho ishowHowInWhenWherewhere didThe was theOnwhenWhy long does

In [61]:
print("Predictions:", predictions)
print("Actual Questions:", actual_questions)

Predictions: ['WhatWhichwhatWho ishowHowInWhenWherewhere didThe was theOnwhenWhy long does much of manyTowhich arewhyAAccording what a kindOf', 'WhatWhichwhatWho ishowHowInWhenWherewhere didThe was theOnwhenWhy long does much of manyTowhich arewhyAAccording what a kindOf', 'WhatWhichwhatWho ishowHowInWhenWherewhere didThe was theOnwhenWhy long does much of manyTowhich arewhyAAccording what a kindOf', 'whatWhatWhichWho ishowHowInWhenWherewhere didThe was theOnwhenWhy long does much of manyTowhich arewhyAAccording what a kindOf', 'WhatwhatWhichWho ishowHowInWhenWherewhere didThe was theOnwhenWhy long does much of manyTowhich arewhyAAccording what a kindOf', 'WhatwhatWhichWho ishowHowInWhenWherewhere didThe was theOnwhenWhy long does much of manyTowhich arewhyAAccording what a kindOf', 'WhatWhichwhatWho ishowHowInWhenWherewhere didThe was theOnwhenWhy long does much of manyTowhich arewhyAAccording what a kindOf', 'whatWhatWhichWho ishowHowInWhenWherewhere didThe was theOnwhenWhy long does

In [62]:
print(type(predictions))
len(predictions)

<class 'list'>


1500

In [63]:
import nltk
from nltk.translate.bleu_score import SmoothingFunction, sentence_bleu
from nltk.translate.meteor_score import meteor_score
from rouge import Rouge

# Ensure the nltk resources are downloaded
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/erap/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [64]:
def calculate_scores(actuals, predictions):
    # Initialize Rouge
    rouge = Rouge()
    smoothie = SmoothingFunction().method1  # Using smoothing method 1

    bleu_scores = []
    rouge_scores = []
    meteor_scores = []

    for actual, prediction in zip(actuals, predictions):
        # Tokenize the sentences
        reference_tokens = actual.split()  # Tokenized reference
        prediction_tokens = prediction.split()  # Tokenized prediction
        
        # Compute BLEU score
        try:
            bleu_score = sentence_bleu([reference_tokens], prediction_tokens, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothie)
        except ZeroDivisionError:
            bleu_score = 0
        
        # Compute ROUGE scores
        rouge_score = rouge.get_scores(prediction, actual)[0]
        
        # Compute METEOR score
        meteor_score_value = meteor_score([reference_tokens], prediction_tokens)  # Tokenized input for METEOR

        bleu_scores.append(bleu_score)
        rouge_scores.append(rouge_score)
        meteor_scores.append(meteor_score_value)
    
    # Average the scores
    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    avg_rouge_1 = sum([score['rouge-1']['f'] for score in rouge_scores]) / len(rouge_scores)
    avg_rouge_2 = sum([score['rouge-2']['f'] for score in rouge_scores]) / len(rouge_scores)
    avg_rouge_l = sum([score['rouge-l']['f'] for score in rouge_scores]) / len(rouge_scores)
    avg_meteor = sum(meteor_scores) / len(meteor_scores)

    return avg_bleu, {'rouge-1': avg_rouge_1, 'rouge-2': avg_rouge_2, 'rouge-l': avg_rouge_l}, avg_meteor


In [65]:
# # Example usage
# actual_questions = ['What form pension is reported?', 'What is the symbol used for kilograms?', 'Who fought for Thunderball the film rights?', "Who can be affected by occupational hazards, that normally don't have an occupation themselves?", 'What is geritol what is geritol', 'What is the Arabic term when referring to Christians in a political sense?', 'Why was then renamed the Kanye West Foundation to in 2008?', 'how tall was lebron james parents', 'what is organophosphate', 'Who was not the Messiah, Jesus or God?', 'What is the force that maintains angular momentum without an unbalanced torque?', 'Sensory deprivation is what is sensory deprivation.', 'What is called what when a dog has a camouflage pattern on her coat?', 'How many times are you supposed to be able to bench press your weight?']
# predictions = ['What is the pension report?', 'What is the symbol used for kilograms in pounds?', 'Who fought over the film rights for Thunderball?', 'Which of the following is not a social issue: teachers, students, or administrators?', 'what is geritol', 'What is the Arabic term when it comes to Christians?', "What was Kanye West's mother's foundation renamed to in 2008?", 'What is Lebron james parents height?', 'what is an organophosphate', 'Who did Luther believe was ultimately responsible for achieving forgiveness?', "What is a consequence of Newton's First Law of Motion that ensures that all bodies maintain their angular momentum?", 'What is sensory deprivation?', 'What is called when a dog has a camouflage pattern on its coat?', 'how many times a man can bench press', 'What is important in the transport of O2 gas?', 'Which of the following people holds the title of chess grandmaster?']

bleu, rouge, meteor = calculate_scores(actual_questions, predictions)
print("BLEU Score:", bleu)
print("ROUGE Scores:", rouge)
print("METEOR Score:", meteor)

BLEU Score: 0.010458954970176815
ROUGE Scores: {'rouge-1': 0.08861184729766776, 'rouge-2': 0.000932342677562869, 'rouge-l': 0.07587090299175533}
METEOR Score: 0.07496914541766228


In [66]:
# cal bert score
from evaluate import load
bertscore = load("bertscore")
#predictions = ["hello there", "general kenobi"]
#references = ["hello there", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=actual_questions, lang="en")
print("Precision:", results['precision'])
print("Recall:", results['recall'])
print("F1 Score:", results['f1'])

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precision: [0.7412158250808716, 0.7211182117462158, 0.7321386933326721, 0.7340778112411499, 0.7419458627700806, 0.7407697439193726, 0.7440854907035828, 0.7402869462966919, 0.7441267967224121, 0.745151162147522, 0.7306728363037109, 0.740843653678894, 0.7411190867424011, 0.7552415132522583, 0.7343026399612427, 0.7395099401473999, 0.7464808225631714, 0.7386118173599243, 0.7384852170944214, 0.737964391708374, 0.7449610829353333, 0.7373532652854919, 0.7419417500495911, 0.7543120980262756, 0.7426550388336182, 0.7439055442810059, 0.7364561557769775, 0.7417958378791809, 0.7331138849258423, 0.7407724857330322, 0.7480597496032715, 0.7480584383010864, 0.7451697587966919, 0.738502025604248, 0.7483677864074707, 0.7342376112937927, 0.7386024594306946, 0.7432069778442383, 0.7325090169906616, 0.7393873333930969, 0.734113335609436, 0.7437201738357544, 0.7314701080322266, 0.7424006462097168, 0.7340024709701538, 0.7367029190063477, 0.7426307201385498, 0.741856038570404, 0.7315400242805481, 0.744667172431

In [67]:
mean_precision = np.mean(results['precision'])
mean_recall = np.mean(results['recall'])
mean_f1 = np.mean(results['f1'])

print("Average Precision:", mean_precision)
print("Average Recall:", mean_recall)
print("Average F1 Score:", mean_f1)

Average Precision: 0.7395917877356212
Average Recall: 0.8018881761233012
Average F1 Score: 0.769405188202858


In [68]:
from tqdm import tqdm

# Initialize empty lists to store contexts and answers
contexts_QG = []
answers_QG = []

# Iterate over each row in the DataFrame
for _, row in tqdm(valid_data.iterrows()):
    # Append the context and the answer text to their respective lists
    contexts_QG.append(row['context'])
    answers_QG.append(row['answer'])  # Assuming 'answer' is the correct column name based on your DataFrame

# Ensure the lists are populated
print(f"Sample context: {contexts_QG[0]}")
print(f"Sample answer: {answers_QG[0]}")

1500it [00:00, 23979.33it/s]

Sample context: Purebred dogs of one breed are genetically distinguishable from purebred dogs of other breeds, but the means by which kennel clubs classify dogs is unsystematic. Systematic analyses of the dog genome has revealed only four major types of dogs that can be said to be statistically distinct. These include the "old world dogs" (e.g., Malamute and Shar Pei), "Mastiff"-type (e.g., English Mastiff), "herding"-type (e.g., Border Collie), and "all others" (also called "modern"- or "hunting"-type).
Sample answer: unsystematic.


In [ ]:
# cal rquge score

rqugescore = load("alirezamsh/rquge")
results = rqugescore.compute(generated_questions=predictions, contexts=contexts_QG, answers= answers_QG)
print("Mean RQUGE Score:", results["mean_score"])


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
max_length = max(len(context) for context in contexts_QG)
print("The maximum length of contexts in contexts_QG is:", max_length)